In [ ]:
!pip install wikipedia-api

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-cp37-none-any.whl size=13462 sha256=57f4c1971353b83c579f8fe65255563e56572c7211890945b3f40710a8bd978a
  Stored in directory: /root/.cache/pip/wheels/bf/40/42/ba1d497f3712281b659dd65b566fc868035c859239571a725a
Successfully built wikipedia-api


In [ ]:
import wikipediaapi
import re

wiki_ru = wikipediaapi.Wikipedia('ru')
wiki_en = wikipediaapi.Wikipedia('en')
all_cat_links = set()


my_list = ['по странам', 'по векам', 'по алфавиту', 'по годам', 'Книги о', 'по музеям', 'Незавершённые', 'по жанрам', \
           'по направлениям', 'по персоналиям', 'по собраниям','по сюжетам', 'по художникам', 'по авторам', \
          'по видам искусства', 'по оконченным вузам', 'по городам', 'по основам изображений', 'по тематике', \
           'по техникам изображения', 'по учителям', 'по эпохам', 'по художнникам']


def link_exists_en(page_name):
    """
    Вспомогательная функция, которая принимает на вход имя страницы Википедии на русском,
    на выходе выдает False, если связанная страница на Викепедии на англиском языке не существет,
    если существует, то возращает её имя
    """
    page_ru = wiki_ru.page(page_name)
    try:
        page_en = page_ru.langlinks['en'].title
        # page_en = pa ge_en.fullurl
    except KeyError as e:
       # print('KeyError',e)
        page_en = False
    return page_en

def get_links_from_category(catmem, level=0, max_level=3):
    """
    На вход принимает cat.categorymembers
    ((cat = wiki_wiki.page("Категория:Живопись")
    y = print_categorymembers(cat.categorymembers)))
    Возращает множество кортежей типа (имя_страницы на русском, имя_страницы на английском)
    all_cat_links - множество, которе уже определено за пределами функции
    """
    # categ = wiki_ru.page(cat)
    # catmem = categ.categorymembers
    for c in catmem.values():
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level and not any(y in c.title for y in my_list):
            # print(c.title)
            get_links_from_category(c.categorymembers, level=level + 1, max_level=max_level)
        elif c.namespace == wikipediaapi.Namespace.MAIN:
            # Articles belong to the main namespace of Wikipedia pages
            # it is page => we can get text
            en_link = link_exists_en(c.title)
            if isinstance(en_link, str):
                all_cat_links.add((c.title, en_link))          
    return all_cat_links
    
def get_links_from_page(page_name):
    """
    Собирает все ссылки с одной страницы на другие страницы, проверяет их наличие на английском языке
    Затем удаляются ссылки которые ведут на страницы про Википедию, про года, дни, века
    Возращаем множество кортежей (имя_страницы на русском, имя_страницы на английском)
    """
    page = wiki_ru.page(page_name)
    all_links = set()
    pattern = re.compile('\d{4}|((Dec|Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov).*)|\d{1,2}.{2}_century|Wikipedia.*|Help:Footnotes')
    links = page.links # словарь название(str) : wikipediaapi.WikipediaPage
    for title in links.keys():
        en_link = link_exists_en(title)
        if isinstance(en_link, str):
            # title_en = re.search('(?<=wiki/)(.+)', en_link).group()
            # print(en_link)
            if  len(re.findall(pattern, en_link)) == 0:
                all_links.add((links[title].title, en_link))
                
    return all_links

def links_to_text(links, cat):
    """
    Преобразует полученный на вход список ссылок в тестовые документы
    В название каждого документа стоит язык, категория и номер (en_tennis_2.txt)
    Эта же страница на другом языке будет записана как ru_tennis_2.txt
    """
    i = 0
    for link in links:
        i += 1
        with open(f'ru_{cat}_{i}.txt', 'w', encoding='utf-8') as f:
            text_ru = wiki_ru.page(link[0]).text
            f.write(text_ru)
        with open(f'en_{cat}_{i}.txt', 'w', encoding='utf-8') as f:        
            text_en = wiki_en.page(link[1]).text
            f.write(text_en)
            
    return f'{i} files were processed'

In [ ]:
# 1 - сначала получаем список страниц из категории
# 2 - потом пускаем цикл по ним и достаем ссылки со страниц (если не это надо)
# 3 - потои превращаем все это в тектстовые документы
# 1
cat = wiki_ru.page("Категория:Живопись")
cat_links = get_links_from_category(cat.categorymembers)
# 2
all_l = set()
for x in cat_links:
    all_l = all_l.union((get_links_from_page(x[0])))
# 3 
all_links = all_l.union(cat.links)
res = links_to_text(all_linls, 'paint')